In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## Importing Basic Packages

import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', 50)

# Importing Plotly Packages
!pip install plotly --upgrade
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

# Importing sklearn Packages

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

# Importing scipy Packages
from scipy.sparse.linalg import svds

In [7]:
final_reviews = pd.read_csv('/content/drive/MyDrive/elite_useful10_process_keyword.csv')
final_reviews.drop(['Unnamed: 0'], axis=1, inplace = True)
print(final_reviews.shape)
final_reviews.head(3)

(62607, 10)


,user_id,business_id,stars,useful,text,processed,sentiment,Dominant_Topic,Keywords,Text_Chunks
0,--2vR0DIsmQ6WfcSzKWigw,nk7fI9Vj7L-kKdXOB7cKIw,5,107,Auf unserem Spaziergang durch das historische ...,auf unserem spaziergang durch das historische ...,1,4,"room, area, parking, year, floor, park, hotel,...","['unserem', 'durch', 'auch', 'irish', 'memoria..."
1,--2vR0DIsmQ6WfcSzKWigw,YiEBSn8VWhnj79igNkShQw,4,112,Auf unserem Rundgang durch das historische Phi...,auf unserem rundgang durch da historische phil...,1,3,"call, tell, time, ask, day, service, work, giv...","['unserem', 'durch', 'sind', 'auch', 'die', 'f..."
2,--2vR0DIsmQ6WfcSzKWigw,Mkr1Bm0SFo51L5-swly0bw,5,110,Auf unserem Weg durch die Altstadt von Philade...,auf unserem weg durch die altstadt von philade...,1,4,"room, area, parking, year, floor, park, hotel,...","['unserem', 'durch', 'die', 'statue', 'gestoße..."


In [9]:
final_reviews['super_score']=final_reviews['stars']

In [10]:
final_reviews.head(3)

,user_id,business_id,stars,useful,text,processed,sentiment,Dominant_Topic,Keywords,Text_Chunks,super_score
0,--2vR0DIsmQ6WfcSzKWigw,nk7fI9Vj7L-kKdXOB7cKIw,5,107,Auf unserem Spaziergang durch das historische ...,auf unserem spaziergang durch das historische ...,1,4,"room, area, parking, year, floor, park, hotel,...","['unserem', 'durch', 'auch', 'irish', 'memoria...",5
1,--2vR0DIsmQ6WfcSzKWigw,YiEBSn8VWhnj79igNkShQw,4,112,Auf unserem Rundgang durch das historische Phi...,auf unserem rundgang durch da historische phil...,1,3,"call, tell, time, ask, day, service, work, giv...","['unserem', 'durch', 'sind', 'auch', 'die', 'f...",4
2,--2vR0DIsmQ6WfcSzKWigw,Mkr1Bm0SFo51L5-swly0bw,5,110,Auf unserem Weg durch die Altstadt von Philade...,auf unserem weg durch die altstadt von philade...,1,4,"room, area, parking, year, floor, park, hotel,...","['unserem', 'durch', 'die', 'statue', 'gestoße...",5


##Collaborative Filtering Recommendation System

Collaborative filtering is a technique that can filter out items that a user might like on the basis of reactions by similar users.

It works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user. It looks at the items they like and combines them to create a ranked list of suggestions.

###Creating User Item Matrix

In [11]:
final_df = final_reviews[['business_id', 'user_id', 'super_score', 'Keywords']]
final_df.head(3)

,business_id,user_id,super_score,Keywords
0,nk7fI9Vj7L-kKdXOB7cKIw,--2vR0DIsmQ6WfcSzKWigw,5,"room, area, parking, year, floor, park, hotel,..."
1,YiEBSn8VWhnj79igNkShQw,--2vR0DIsmQ6WfcSzKWigw,4,"call, tell, time, ask, day, service, work, giv..."
2,Mkr1Bm0SFo51L5-swly0bw,--2vR0DIsmQ6WfcSzKWigw,5,"room, area, parking, year, floor, park, hotel,..."


In [12]:
# Load the Yelp business dataset
df = pd.read_json('/content/drive/MyDrive/yelp_academic_dataset_business.json', lines=True)
df_cleaned = df.dropna()
filtered_businesses = df_cleaned[(df_cleaned['state'] == 'AZ') & (df_cleaned['city'].notnull()) & (df_cleaned['is_open'] == 1) & (df_cleaned['stars'] >= 3.5)]

keywords = [
    'Restaurants', 'Cafes', 'Diners', 'Bistros', 'Food', 'Bars',
    'Italian', 'Mexican', 'Chinese', 'Japanese', 'Thai', 'Indian',
    'French', 'Vietnamese', 'Greek', 'Lebanese', 'Korean', 'American',
    'Mediterranean', 'Spanish', 'Seafood', 'Sushi', 'Pizza', 'Bakery',
    'Coffee & Tea', 'Specialty Food', 'Ice Cream', 'Fast Food', 'BBQ',
    'Steakhouse', 'Vegan', 'Vegetarian', 'Gluten-Free'
]

# Adjusting the filtering to include a broad spectrum of dining and food-related establishments
az_restaurant = filtered_businesses[filtered_businesses['categories'].str.contains('|'.join(keywords), case=False, na=False)]

In [13]:
az_restaurant.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
132,9C2rpb56aQvW0ViZHK9sPw,Home Plate Sports Pub,4880 E 22nd St,Tucson,AZ,85711,32.206315,-110.887843,3.5,56,1,"{'RestaurantsGoodForGroups': 'True', 'Alcohol'...","Bars, Restaurants, Sports Bars, Nightlife","{'Monday': '10:0-2:0', 'Tuesday': '10:0-2:0', ..."
234,Y6heWJJ9AmEL58fZwgi9YQ,Rosati's Pizza,2944 W Ina Rd,Tucson,AZ,85741,32.338724,-111.030310,4.0,58,1,"{'RestaurantsDelivery': 'True', 'RestaurantsTa...","Sports Bars, Bars, Nightlife, Italian, Restaur...","{'Monday': '0:0-0:0', 'Tuesday': '10:30-0:0', ..."
319,f82dhKNiUXsDVPMLqKYiIQ,Sher-e-Punjab,853 East Grant Rd,Tucson,AZ,85719,32.250960,-110.959158,4.0,446,1,"{'RestaurantsAttire': ''casual'', 'BusinessAcc...","Restaurants, Salad, Pakistani, Indian, Cocktai...","{'Tuesday': '16:0-21:0', 'Wednesday': '16:0-21..."
457,D7z_V6E8cxXghSRcHAmUPg,La Bella China Restaurant,5680 S 12th Ave,Tucson,AZ,85706,32.149473,-110.978186,4.0,92,1,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Restaurants, Chinese","{'Tuesday': '11:0-19:30', 'Wednesday': '11:0-1..."
553,adATTqggIQX5xxLDISkFTw,Just Churros,,Tucson,AZ,85705,32.271231,-110.992075,5.0,25,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Food Trucks, Restaurants, Caterers, Event Plan...","{'Monday': '0:0-0:0', 'Friday': '15:0-21:0', '..."


In [14]:
az_restaurant = az_restaurant[['business_id', 'name', 'categories']]
az_restaurant.head()

,business_id,name,categories
132,9C2rpb56aQvW0ViZHK9sPw,Home Plate Sports Pub,"Bars, Restaurants, Sports Bars, Nightlife"
234,Y6heWJJ9AmEL58fZwgi9YQ,Rosati's Pizza,"Sports Bars, Bars, Nightlife, Italian, Restaur..."
319,f82dhKNiUXsDVPMLqKYiIQ,Sher-e-Punjab,"Restaurants, Salad, Pakistani, Indian, Cocktai..."
457,D7z_V6E8cxXghSRcHAmUPg,La Bella China Restaurant,"Restaurants, Chinese"
553,adATTqggIQX5xxLDISkFTw,Just Churros,"Food Trucks, Restaurants, Caterers, Event Plan..."


In [15]:
az_data = pd.merge(final_df, az_restaurant, on='business_id')
az_data.head(5)

,business_id,user_id,super_score,Keywords,name,categories
0,WSx9-iYYyST_umny9sJBFg,-7qUbOVWJp2NT3f_TNuIBA,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A..."
1,WSx9-iYYyST_umny9sJBFg,6ObFF8-uKnOAlXuSH4TlyQ,4,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A..."
2,WSx9-iYYyST_umny9sJBFg,BpJygDyhzDf7Pw1Is_5zSA,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A..."
3,WSx9-iYYyST_umny9sJBFg,CjhTdmN6qWEKC7Zo6qadow,4,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A..."
4,WSx9-iYYyST_umny9sJBFg,Ck5pDKaQenokLE9s5tYNcQ,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A..."


In [16]:
az_data['All_Keywords'] = az_data['categories'].str.cat(az_data['Keywords'],sep=", ")
az_data.head()

,business_id,user_id,super_score,Keywords,name,categories,All_Keywords
0,WSx9-iYYyST_umny9sJBFg,-7qUbOVWJp2NT3f_TNuIBA,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","Gastropubs, Cajun/Creole, Southern, Burgers, A..."
1,WSx9-iYYyST_umny9sJBFg,6ObFF8-uKnOAlXuSH4TlyQ,4,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","Gastropubs, Cajun/Creole, Southern, Burgers, A..."
2,WSx9-iYYyST_umny9sJBFg,BpJygDyhzDf7Pw1Is_5zSA,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","Gastropubs, Cajun/Creole, Southern, Burgers, A..."
3,WSx9-iYYyST_umny9sJBFg,CjhTdmN6qWEKC7Zo6qadow,4,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","Gastropubs, Cajun/Creole, Southern, Burgers, A..."
4,WSx9-iYYyST_umny9sJBFg,Ck5pDKaQenokLE9s5tYNcQ,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","Gastropubs, Cajun/Creole, Southern, Burgers, A..."


In [17]:
az_data.All_Keywords.head(1)

0    Gastropubs, Cajun/Creole, Southern, Burgers, A...
Name: All_Keywords, dtype: object

In [20]:
az_restaurant_rating = az_data.pivot_table(index = 'user_id', columns = 'name', values = 'super_score')
az_restaurant_rating.head()

name,1702,4 Seasons,47 Scott,5 Points Market & Restaurant,99 Cents Only Stores,AJ's Fine Foods,Agustin Kitchen,Alafia West African Cuisine,Alejandro's Tortilla Factory,American Eat Company,Anello,Angry Crab Shack & BBQ,Anita Street Market,Aqui Con El Nene,Arizona Inn,Arizona Pizza Company,BBQ RUSH,BK Tacos,Ba-Dar,Babylon Market,Back Dough,Bai Thong,Baja Cafe,Baja Cafe On Ina,Banhdicted,...,Urban Fresh,Urban Pita,Vero Amore - Swan,Villa Peru,Village Bakehouse,Viva Coffee House,Vivace,Welcome Diner,Whataburger,Whiskey Del Bac,Whole Foods Market,Wild Garlic Grill,Wildflower,Wok & Roll,Woops! Bakeshop Main Gate Square,Yamato,Yellow Brick Coffee,Yoshimatsu Japanese Eatery,Zayna Mediterranean,Zemam's,Zinburger,Zona 78,eHotPot,eegee's,the Glowbal
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-7qUbOVWJp2NT3f_TNuIBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-GpmKjPY01mDDR5rsy8CPQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-WYbVngIGDQ81MY12UtwnA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-XFxvFtJomoGoyH3PxYFrQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-dccg5dCgQRh8ObtNvrj3g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Normalizing the Rating Scores

def mean_center_rows(df):
    return (df.T - df.mean(axis = 1)).T

az_restaurant_rating = mean_center_rows(az_restaurant_rating)
az_restaurant_rating.head()

name,1702,4 Seasons,47 Scott,5 Points Market & Restaurant,99 Cents Only Stores,AJ's Fine Foods,Agustin Kitchen,Alafia West African Cuisine,Alejandro's Tortilla Factory,American Eat Company,Anello,Angry Crab Shack & BBQ,Anita Street Market,Aqui Con El Nene,Arizona Inn,Arizona Pizza Company,BBQ RUSH,BK Tacos,Ba-Dar,Babylon Market,Back Dough,Bai Thong,Baja Cafe,Baja Cafe On Ina,Banhdicted,...,Urban Fresh,Urban Pita,Vero Amore - Swan,Villa Peru,Village Bakehouse,Viva Coffee House,Vivace,Welcome Diner,Whataburger,Whiskey Del Bac,Whole Foods Market,Wild Garlic Grill,Wildflower,Wok & Roll,Woops! Bakeshop Main Gate Square,Yamato,Yellow Brick Coffee,Yoshimatsu Japanese Eatery,Zayna Mediterranean,Zemam's,Zinburger,Zona 78,eHotPot,eegee's,the Glowbal
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-7qUbOVWJp2NT3f_TNuIBA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-GpmKjPY01mDDR5rsy8CPQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-WYbVngIGDQ81MY12UtwnA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-XFxvFtJomoGoyH3PxYFrQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-dccg5dCgQRh8ObtNvrj3g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Filling all Null Values with 0.0

az_restaurant_rating = az_restaurant_rating.fillna(0)
az_restaurant_rating.head()

name,1702,4 Seasons,47 Scott,5 Points Market & Restaurant,99 Cents Only Stores,AJ's Fine Foods,Agustin Kitchen,Alafia West African Cuisine,Alejandro's Tortilla Factory,American Eat Company,Anello,Angry Crab Shack & BBQ,Anita Street Market,Aqui Con El Nene,Arizona Inn,Arizona Pizza Company,BBQ RUSH,BK Tacos,Ba-Dar,Babylon Market,Back Dough,Bai Thong,Baja Cafe,Baja Cafe On Ina,Banhdicted,...,Urban Fresh,Urban Pita,Vero Amore - Swan,Villa Peru,Village Bakehouse,Viva Coffee House,Vivace,Welcome Diner,Whataburger,Whiskey Del Bac,Whole Foods Market,Wild Garlic Grill,Wildflower,Wok & Roll,Woops! Bakeshop Main Gate Square,Yamato,Yellow Brick Coffee,Yoshimatsu Japanese Eatery,Zayna Mediterranean,Zemam's,Zinburger,Zona 78,eHotPot,eegee's,the Glowbal
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-7qUbOVWJp2NT3f_TNuIBA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-GpmKjPY01mDDR5rsy8CPQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-WYbVngIGDQ81MY12UtwnA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-XFxvFtJomoGoyH3PxYFrQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-dccg5dCgQRh8ObtNvrj3g,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
az_restaurant_rating.mean(axis=1)

user_id
-7qUbOVWJp2NT3f_TNuIBA    0.0
-GpmKjPY01mDDR5rsy8CPQ    0.0
-WYbVngIGDQ81MY12UtwnA    0.0
-XFxvFtJomoGoyH3PxYFrQ    0.0
-dccg5dCgQRh8ObtNvrj3g    0.0
                         ... 
zESNoyq5QjGgyHnMcGiBiQ    0.0
zQ1OJLGeBLGEcnIPzxlCPA    0.0
zSzgPwvE8BIQQnqvDKBN6A    0.0
zWC1npRbmWO2bZRt2fYgfw    0.0
ztAjPFZezR1Yd7syCQp60Q    0.0
Length: 497, dtype: float64

In [24]:
# Check Mean scores across business_ids

az_restaurant_rating.mean(axis = 0)

name
1702                            0.000671
4 Seasons                       0.000550
47 Scott                        0.002004
5 Points Market & Restaurant   -0.002356
99 Cents Only Stores            0.000000
                                  ...   
Zinburger                       0.000151
Zona 78                         0.000000
eHotPot                         0.001006
eegee's                        -0.005048
the Glowbal                     0.000000
Length: 393, dtype: float64

###Cosine Similarity

The cosine similarity between the user profile and all item profiles is computed to establish each user's preference. Cosine similarity measures the similarity between two vectors by calculating the cosine of the angle between them. It is the dot product of two vectors divided by the product of the magnitude of each vector. As the dot product is divided by the magnitude, data is normalised and only the angle difference is measured. The cosine similarity of each restaurant to each user is calculated. The cosine of a 0 degree angle is 1, therefore the closer to 1 the cosine similarity is, the more similar the items are.

In [25]:
print(az_restaurant_rating.shape)
az_restaurant_rating

(497, 393)


name,1702,4 Seasons,47 Scott,5 Points Market & Restaurant,99 Cents Only Stores,AJ's Fine Foods,Agustin Kitchen,Alafia West African Cuisine,Alejandro's Tortilla Factory,American Eat Company,Anello,Angry Crab Shack & BBQ,Anita Street Market,Aqui Con El Nene,Arizona Inn,Arizona Pizza Company,BBQ RUSH,BK Tacos,Ba-Dar,Babylon Market,Back Dough,Bai Thong,Baja Cafe,Baja Cafe On Ina,Banhdicted,...,Urban Fresh,Urban Pita,Vero Amore - Swan,Villa Peru,Village Bakehouse,Viva Coffee House,Vivace,Welcome Diner,Whataburger,Whiskey Del Bac,Whole Foods Market,Wild Garlic Grill,Wildflower,Wok & Roll,Woops! Bakeshop Main Gate Square,Yamato,Yellow Brick Coffee,Yoshimatsu Japanese Eatery,Zayna Mediterranean,Zemam's,Zinburger,Zona 78,eHotPot,eegee's,the Glowbal
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-7qUbOVWJp2NT3f_TNuIBA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-GpmKjPY01mDDR5rsy8CPQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-WYbVngIGDQ81MY12UtwnA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-XFxvFtJomoGoyH3PxYFrQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-dccg5dCgQRh8ObtNvrj3g,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zESNoyq5QjGgyHnMcGiBiQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zQ1OJLGeBLGEcnIPzxlCPA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zSzgPwvE8BIQQnqvDKBN6A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Converting the Matrix DataFrame into a NumPy array

az_matrix = az_restaurant_rating.to_numpy()
az_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

###Truncated SVD

When it comes to matrix factorization technique, truncated Singular Value Decomposition (SVD) is a popular method to produce features that factors a matrix M into the three matrices U, Σ, and V. Truncated SVD shares similarity with Principal Component Analysis while SVD is produced from the data matrix and the factorization of PCA is generated from the covariance matrix.

Unlike regular SVDs, truncated SVD produces a factorization where the number of columns can be specified for a number of truncation. For example, given an n x n matrix, truncated SVD generates the matrices with the specified number of columns, whereas SVD outputs n columns of matrices. Truncated SVD can deal with sparse matrix to generate features’ matrices, whereas PCA would operate on the entire matrix for the output of the covariance matrix.

In [27]:
# Applying Singular Value Decomposition (SVD)

#The number of factors to factor the user-item matrix.

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix

U, sigma, Vt = svds(az_matrix, k = NUMBER_OF_FACTORS_MF)

print(U.shape)
print(Vt.shape)

sigma = np.diag(sigma)
sigma.shape

(497, 15)
(15, 393)


(15, 15)

After the factorization, we try to to reconstruct the original matrix by multiplying its factors. The resulting matrix is not sparse any more. It was generated predictions for items the user have not yet interaction, which we will exploit for recommendations.

In [30]:
# List of first 10 Yelp Customer User_ids in the Matrix

user_ids = list(az_restaurant_rating.index)


In [28]:
# Overview of user ratings across all Restaurants in Arizona

all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[ 2.35513669e-18,  4.03977270e-18, -8.11501838e-19, ...,
         2.22751116e-20, -2.13982085e-17,  4.42701905e-33],
       [ 2.90753174e-18,  2.87454463e-18, -1.18240817e-18, ...,
         5.36117110e-22, -2.74877747e-18,  5.83711670e-34],
       [ 9.22188696e-18,  1.01092845e-17, -7.77755412e-19, ...,
         1.24480289e-20, -5.00801436e-19,  2.92945118e-33],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [31]:
# Converting the reconstructed matrix back to a Pandas dataframe

cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = az_restaurant_rating.columns, index=user_ids).transpose()
print(cf_preds_df.shape)
cf_preds_df

(393, 497)


,-7qUbOVWJp2NT3f_TNuIBA,-GpmKjPY01mDDR5rsy8CPQ,-WYbVngIGDQ81MY12UtwnA,-XFxvFtJomoGoyH3PxYFrQ,-dccg5dCgQRh8ObtNvrj3g,-rGU3wo4fgEnGRboYdMTjw,-sC90qgp5JQmF_-59x_mVg,-wd41HcxuwQwZJRW9HoO1A,0-rEsSSEvV9eEBGCt1Fl4w,064Z_GuIr54fgJdmDXqnGA,0Hl2BwAIz3Gi3Jhlx7AYZA,0JiIRjH5WM8qFs8iO3Tang,0SMx6NDTWmS51FNP5QZApg,0cHEkFahUqJiobK3vo8BbA,0juzbrgcLHqobdK-OZAyiw,0yrF14nWVAFeI0fG29a63g,1F5wbSbtUBzxS8IsM7fWsA,1GgldkdtUkRfl3_84w3-Aw,1ZfTuwipogg1pL4ICvHHoQ,1deDzlWUCW8JUhODGBbB5w,1k4GFvMYN6678nhe8eGqlA,1sGYXSkJHPhJ6wQtc-RbZw,22G779jY-UgMWN8msYkV4Q,27wY36k_bVHmEbKoLkh1wg,2ASSfvbV7lPeqbQlWQgMVA,...,vsfPTfNV_K_s02tH6ZCHxA,w9OZUPIoweoUYVbl1RwCMQ,w9UN4BVilzIPEdroPGh6KA,wTfb2nfzPIyFcYQArdPtuQ,whLn2WLjAad7sBkcog-wsA,wrFEiYUhHi1toO7kJeqJ3g,x15nXGkbW58Lf8urQHewpg,xMPHgwla-oQeNlWmE_Wnag,x_FhrxfP4gbu7fI4Zypylg,xftrD59docmsj98QISmbow,xndFXZ7YDsszg9osLjZvIA,yEYE08qmSdnwUl00KiiRFA,yMh4OF_Li2gJzE5rJ8oCbA,yQ37GLiWfj7hehzZQxS86w,yU4MW48QQ1eAdOkztAVf3g,yWWd4KTDNuiNsP1ecA5sVQ,y_nRpTpmsteDzhdGKLnXiw,yhrTOO2A4hQEMWOI3pFcSA,z33x0fHD8gutPy5Gv6Hwpw,zBWSORvlWDmwtcx6627qWQ,zESNoyq5QjGgyHnMcGiBiQ,zQ1OJLGeBLGEcnIPzxlCPA,zSzgPwvE8BIQQnqvDKBN6A,zWC1npRbmWO2bZRt2fYgfw,ztAjPFZezR1Yd7syCQp60Q
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1702,2.355137e-18,2.907532e-18,9.221887e-18,-1.321960e-18,2.012167e-19,1.519923e-18,1.329902e-17,6.912176e-18,-2.325319e-18,2.782532e-19,6.142318e-03,-5.057873e-20,1.391489e-19,-2.295873e-20,-8.756933e-05,0.0,3.594211e-04,0.0,1.570974e-04,0.0,0.0,-1.042984e-03,0.0,0.0,0.0,...,2.426925e-03,0.0,0.0,4.581876e-03,-9.724723e-04,-4.405860e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.541897e-03,0.0,0.0,0.0,0.0,8.804685e-04,0.0,1.086126e-03,0.0,0.0,0.0
4 Seasons,4.039773e-18,2.874545e-18,1.010928e-17,-6.715727e-19,1.262863e-18,2.475295e-18,2.338458e-17,3.086992e-18,-2.972313e-18,-9.101498e-18,8.906472e-03,2.972062e-18,-8.574010e-18,1.413564e-18,-2.855784e-03,0.0,6.468675e-04,0.0,2.320081e-03,0.0,0.0,-2.369087e-03,0.0,0.0,0.0,...,1.035155e-02,0.0,0.0,8.535784e-03,-1.304748e-03,-3.959595e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.631495e-02,0.0,0.0,0.0,0.0,1.286866e-03,0.0,9.415781e-03,0.0,0.0,0.0
47 Scott,-8.115018e-19,-1.182408e-18,-7.777554e-19,3.302374e-18,-7.638631e-19,-3.115495e-18,-9.811158e-18,-6.081693e-19,3.178535e-18,6.698080e-18,-2.152978e-03,-2.880006e-18,6.897017e-18,-1.140148e-18,2.805421e-03,0.0,2.588620e-02,0.0,-1.671907e-03,0.0,0.0,2.836850e-03,0.0,0.0,0.0,...,-1.922087e-03,0.0,0.0,-3.121460e-03,-6.296448e-02,2.493834e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.026288e-02,0.0,0.0,0.0,0.0,6.409347e-02,0.0,-6.249919e-03,0.0,0.0,0.0
5 Points Market & Restaurant,-1.433341e-19,1.112223e-18,3.412024e-19,-3.127593e-18,6.535205e-20,2.091962e-18,2.834274e-18,2.629425e-18,-2.289696e-18,-5.159331e-19,4.108310e-04,7.541595e-19,-1.044484e-18,1.746544e-19,-8.093681e-04,0.0,-2.095290e-02,0.0,2.316372e-04,0.0,0.0,1.606309e-04,0.0,0.0,0.0,...,-2.664232e-03,0.0,0.0,5.464754e-04,5.441308e-02,-1.637086e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.958146e-04,0.0,0.0,0.0,0.0,-5.379696e-02,0.0,6.818678e-04,0.0,0.0,0.0
99 Cents Only Stores,-1.558417e-33,-1.293699e-34,-9.323118e-34,-1.930715e-34,-2.221543e-35,7.960012e-34,1.892647e-33,1.047787e-33,-1.414975e-33,-2.304360e-34,-6.754044e-19,1.316795e-34,1.277731e-34,-1.982286e-35,-2.792934e-19,0.0,-1.349238e-17,0.0,-1.021514e-19,0.0,0.0,5.930802e-18,0.0,0.0,0.0,...,-3.060212e-18,0.0,0.0,-5.398929e-19,1.662541e-18,1.152097e-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.861775e-17,0.0,0.0,0.0,0.0,-1.121035e-17,0.0,9.734656e-18,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zinburger,-2.663383e-20,2.324156e-19,3.923885e-19,1.980013e-18,3.383887e-20,-1.612738e-18,1.455044e-19,-3.164726e-19,1.976277e-18,1.068712e-18,-8.384782e-04,-4.285375e-18,-9.851894e-20,-1.078457e-20,6.954345e-03,0.0,1.987059e-01,0.0,-6.252104e-05,0.0,0.0,-6.583244e-04,0.0,0.0,0.0,...,-8.814918e-03,0.0,0.

In [32]:
# Creating Item-Item Matrix based on Cosine Similarity

item_item_matrix = cosine_similarity(cf_preds_df)
item_item_matrix= pd.DataFrame(item_item_matrix, columns=cf_preds_df.index, index = cf_preds_df.index)
item_item_matrix

name,1702,4 Seasons,47 Scott,5 Points Market & Restaurant,99 Cents Only Stores,AJ's Fine Foods,Agustin Kitchen,Alafia West African Cuisine,Alejandro's Tortilla Factory,American Eat Company,Anello,Angry Crab Shack & BBQ,Anita Street Market,Aqui Con El Nene,Arizona Inn,Arizona Pizza Company,BBQ RUSH,BK Tacos,Ba-Dar,Babylon Market,Back Dough,Bai Thong,Baja Cafe,Baja Cafe On Ina,Banhdicted,...,Urban Fresh,Urban Pita,Vero Amore - Swan,Villa Peru,Village Bakehouse,Viva Coffee House,Vivace,Welcome Diner,Whataburger,Whiskey Del Bac,Whole Foods Market,Wild Garlic Grill,Wildflower,Wok & Roll,Woops! Bakeshop Main Gate Square,Yamato,Yellow Brick Coffee,Yoshimatsu Japanese Eatery,Zayna Mediterranean,Zemam's,Zinburger,Zona 78,eHotPot,eegee's,the Glowbal
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1702,1.000000e+00,5.179207e-01,-2.573171e-02,8.210707e-02,-2.599735e-17,-7.985988e-17,-8.062668e-02,-2.906944e-03,2.800887e-03,2.585598e-03,2.585598e-03,-1.088806e-03,2.585598e-03,2.386282e-03,-3.427416e-03,-3.844508e-03,3.570557e-19,-9.070998e-02,-1.354887e-02,2.728544e-02,7.628721e-17,9.070998e-02,1.088806e-03,1.659243e-17,1.634903e-17,...,2.585598e-03,-1.098411e-02,-1.182069e-16,4.033860e-18,2.585598e-03,3.287411e-17,-1.487419e-02,9.994901e-01,-5.357478e-17,2.743320e-03,3.170332e-01,-7.460811e-04,-5.484748e-03,1.059605e-03,-5.667105e-18,-8.337853e-01,2.585598e-03,-1.298957e-16,-2.585598e-03,-9.070998e-02,2.039112e-03,-9.634063e-18,1.120012e-01,-4.462121e-04,8.566424e-17
4 Seasons,5.179207e-01,1.000000e+00,-8.260884e-01,4.518324e-01,-2.293809e-17,9.598647e-17,-5.045709e-02,-1.978806e-03,2.072689e-03,-8.537429e-01,-8.537429e-01,-9.921595e-04,-8.537429e-01,-8.535750e-01,8.504180e-01,-5.755815e-02,5.513840e-20,-6.647213e-02,-1.020290e-02,1.158054e-01,5.974397e-17,6.647213e-02,9.921595e-04,3.666822e-18,2.051806e-18,...,-8.537429e-01,-4.904230e-02,-4.214719e-17,6.393181e-19,-8.537429e-01,9.023799e-18,-5.004553e-03,5.243251e-01,-8.046170e-18,3.238079e-02,8.206959e-02,9.990714e-05,-4.637726e-03,7.167275e-04,-6.011045e-19,-9.039175e-01,-8.537429e-01,-3.240530e-17,8.537429e-01,-6.647213e-02,6.875579e-03,-1.831499e-19,8.374836e-01,-4.812332e-04,1.871316e-17
47 Scott,-2.573171e-02,-8.260884e-01,1.000000e+00,-7.276872e-01,6.065691e-18,-1.757228e-16,3.190135e-01,-1.355223e-01,1.454993e-01,9.505802e-01,9.505802e-01,-5.596038e-02,9.505802e-01,9.501134e-01,-9.483931e-01,7.514999e-02,2.142333e-19,3.322893e-01,1.034694e-02,-1.176322e-01,3.088678e-17,-3.322893e-01,5.596038e-02,8.423277e-18,9.696873e-18,...,9.505802e-01,3.600972e-02,-4.127589e-17,2.151428e-18,9.505802e-01,1.493960e-17,-1.952751e-02,-2.712021e-02,-3.198853e-17,-5.269360e-02,1.269542e-01,5.157684e-02,-2.092064e-01,-1.569231e-01,-3.618161e-18,5.462979e-01,9.505802e-01,-6.155492e-17,-9.505802e-01,3.322893e-01,3.923660e-02,-7.596699e-18,-8.506118e-01,-1.080097e-02,4.110566e-17
5 Points Market & Restaurant,8.210707e-02,4.518324e-01,-7.276872e-01,1.000000e+00,6.543701e-18,1.373542e-16,-8.779084e-01,3.943102e-01,-4.113488e-01,-4.789024e-01,-4.789024e-01,1.525674e-01,-4.789024e-01,-4.782036e-01,4.802772e-01,-8.252467e-02,-2.382842e-19,-8.883176e-01,-2.204984e-02,7.258826e-02,-1.465939e-16,8.883176e-01,-1.525674e-01,-9.758539e-18,-9.659223e-18,...,-4.789024e-01,9.013613e-03,5.827782e-17,-2.112293e-18,-4.789024e-01,-1.833617e-17,3.925128e-02,6.902903e-02,3.444997e-17,6.468493e-02,-1.445273e-01,-1.475541e-01,5.886679e-01,4.416169e-01,3.679881e-18,-3.329744e-01,-4.789024e-01,7.399763e-17,4.789024e-01,-8.883176e-01,-1.240175e-01,7.869949e-18,3.969474e-01,3.419353e-02,-4.369815e-17
99 Cents Only Stores,-2.599735e-17,-2.293809e-17,6.065691e-18,6.543701e-18,3.392036e-33,4.644301e-33,-1.410120e-17,1.716599e-18,-1.181315e-17,1.093171e-17,1.093171e-17,1.353943e-17,1.093171e-17,1.088993e-17,-1.299788e-17,2.437419e-17,-4.053206e-35,-1.367712e-17,-2.049562e-18,-2.625033e-18,-4.654981e-33,1.367712e-17,-1.353943e-17,-1.873540e-33,-1.707047e-33,...,1.093171e-17,-2.439618e-17,1.277406e-32,-3.883614e-34,1.093171e-1

In [33]:
# Creating Collaborative Filtering Function for Restaurant-Restaurant Recommendation System

def cf_recommender(restaurant):

    """Getting the correlation of a specific restaurant with other Toronto Restaurants"""
    restaurant_ratings = cf_preds_df.T[restaurant]
    similar_restaurant_ratings = cf_preds_df.T.corrwith(restaurant_ratings)
    corr_ratings = pd.DataFrame(similar_restaurant_ratings, columns=['Correlation'])
    corr_ratings.dropna(inplace=True)

    """Retrieving the Ratings Scores from the Item-Item Matrix"""
    ratings_sim = item_item_matrix[restaurant]

    """Filtering for positively correlated restaurants"""
    ratings_sim = ratings_sim[ratings_sim>0]

    """Generate Top 10 Recommended Restaurants"""
    """Exclude top row as that will be the same restaurant"""
    return ratings_sim.sort_values(ascending= False).head(11)[1:]

Let's test out our Collaborative Filtering Model on 2 Restaurants: 5 Points Market & Restaurant, and eHotPot.

In [34]:
# Top 10 Recommended Restaurants that people should visit if they had previously visited 5 Points Market & Restaurant

restaurant_recommendations = cf_recommender('5 Points Market & Restaurant')
restaurant_recommendations = pd.DataFrame(data = restaurant_recommendations)
restaurant_recommendations

,5 Points Market & Restaurant
name,
Bai Thong,0.888318
IOU Sushi III,0.888318
Kababeque Indian Grill,0.888318
El Gorrion,0.888318
Kazoku,0.887736
Crispy's Fish-n-Chips,0.741549
Sushi Cortaro,0.627089
Wildflower,0.588668
The Keg Steakhouse and Bar - Oro Valley,0.584452


**Comparing the Super Score Ratings and Restaurant Categories between 5 Points Market & Restaurant and the 3 highest recommended restaurants.**

In [37]:
restaurant = az_data[az_data['name'] == '5 Points Market & Restaurant'][['name','categories','super_score']]
restaurant.head()

,name,categories,super_score
342,5 Points Market & Restaurant,"Restaurants, American (New), Delis, Vegetarian...",5
343,5 Points Market & Restaurant,"Restaurants, American (New), Delis, Vegetarian...",4
344,5 Points Market & Restaurant,"Restaurants, American (New), Delis, Vegetarian...",4
345,5 Points Market & Restaurant,"Restaurants, American (New), Delis, Vegetarian...",4
346,5 Points Market & Restaurant,"Restaurants, American (New), Delis, Vegetarian...",2


In [38]:
restaurant_1 = az_data[az_data['name'] == 'Bai Thong'][['name','categories','super_score']]
restaurant_1.head()

,name,categories,super_score
800,Bai Thong,"Thai, Restaurants",4


In [40]:
restaurant_2 = az_data[az_data['name'] == 'IOU Sushi III'][['name','categories','super_score']]
restaurant_2.head()

,name,categories,super_score
803,IOU Sushi III,"Japanese, Restaurants, Sushi Bars, Themed Cafe...",4


In [43]:
# Top 10 Recommended Restaurants that people should visit if they had previously visited eHotPot

restaurant_recommendations = cf_recommender('eHotPot')
restaurant_recommendations = pd.DataFrame(data = restaurant_recommendations)
restaurant_recommendations

,eHotPot
name,
Smokey Mo,1.000000
Sammy's Mexican Grill,0.929853
Heirloom Farmers Market - Oro Valley,0.929853
The Keg Steakhouse and Bar - Oro Valley,0.929853
North Italia,0.914793
HUB Restaurant & Ice Creamery,0.911950
Boca Tacos Y Tequila,0.911715
El Berraco,0.911621
Beaut Burger,0.911621


##Content Based Recommendation System

###Data Cleaning

In [44]:
az_data['All_Keywords'] = az_data['All_Keywords'].map(lambda x: str(x))
az_data['All_Keywords'] = az_data['All_Keywords'].map(lambda x: x.lower())
print(az_data.shape)
az_data.head()

(902, 7)


,business_id,user_id,super_score,Keywords,name,categories,All_Keywords
0,WSx9-iYYyST_umny9sJBFg,-7qUbOVWJp2NT3f_TNuIBA,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","gastropubs, cajun/creole, southern, burgers, a..."
1,WSx9-iYYyST_umny9sJBFg,6ObFF8-uKnOAlXuSH4TlyQ,4,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","gastropubs, cajun/creole, southern, burgers, a..."
2,WSx9-iYYyST_umny9sJBFg,BpJygDyhzDf7Pw1Is_5zSA,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","gastropubs, cajun/creole, southern, burgers, a..."
3,WSx9-iYYyST_umny9sJBFg,CjhTdmN6qWEKC7Zo6qadow,4,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","gastropubs, cajun/creole, southern, burgers, a..."
4,WSx9-iYYyST_umny9sJBFg,Ck5pDKaQenokLE9s5tYNcQ,5,"order, food, restaurant, menu, sauce, eat, fry...",The Parish,"Gastropubs, Cajun/Creole, Southern, Burgers, A...","gastropubs, cajun/creole, southern, burgers, a..."


In [53]:
# Adding and Grouping Rows together by Restaurant Name

az_final = az_data.groupby('name')['All_Keywords'].sum()
az_final = az_final.to_frame(name = 'sum').reset_index()
az_final

,name,sum
0,1702,"beer, wine & spirits, restaurants, sports bars..."
1,4 Seasons,"thai, breakfast & brunch, chinese, american (t..."
2,47 Scott,"restaurants, nightlife, breakfast & brunch, am..."
3,5 Points Market & Restaurant,"restaurants, american (new), delis, vegetarian..."
4,99 Cents Only Stores,"event planning & services, professional servic..."
...,...,...
388,Zinburger,"restaurants, event planning & services, gluten..."
389,Zona 78,"desserts, food, restaurants, pizza, italian, c..."
390,eHotPot,"chinese, taiwanese, hot pot, restaurants, orde..."
391,eegee's,"sandwiches, restaurants, juice bars & smoothie..."


In [54]:
# Getting a list of Unique Keywords per Restaurant

az_final['sum'] = az_final['sum'].map(lambda x: x.replace(", ","', '"))
az_final['sum'] = az_final['sum'].map(lambda x: str("'") + x + str("'"))
f = lambda x: x["sum"].split(", ")
az_final['sum'] = az_final.apply(f, axis=1)
az_final['sum'] = az_final['sum'].map(lambda x: set(x))
az_final.set_index('name', inplace = True)
az_final.head()

,sum
name,
1702,"{'time', 'sports bars', 'back', 'place', 'grea..."
4 Seasons,"{'chicken', 'american (traditional)', 'sauce',..."
47 Scott,"{'american (traditional)', 'mealrestaurants', ..."
5 Points Market & Restaurant,"{'mealrestaurants', 'time', 'back', 'place', '..."
99 Cents Only Stores,"{'event planning & services', 'grocery', 'disc..."


In [55]:
# Creating Bag of Words

az_final['bag_of_words'] = ''
columns = az_final.columns
for index, row in az_final.iterrows():
    words = ''
    for col in columns:
            words = words + ' '.join(row[col])+ ' '
    row['bag_of_words'] = words

az_final.drop(columns = [col for col in az_final.columns if col!= 'bag_of_words'], inplace = True)

az_final.head()

,bag_of_words
name,
1702,'time' 'sports bars' 'back' 'place' 'great' 'l...
4 Seasons,'chicken' 'american (traditional)' 'sauce' 're...
47 Scott,'american (traditional)' 'mealrestaurants' 'ga...
5 Points Market & Restaurant,'mealrestaurants' 'time' 'back' 'place' 'sauce...
99 Cents Only Stores,'event planning & services' 'grocery' 'discoun...


In [56]:
# Remove quotation marks

az_final['bag_of_words'] = az_final['bag_of_words'].map(lambda x: x.replace("'", ""))
az_final.head()

,bag_of_words
name,
1702,time sports bars back place great look drink b...
4 Seasons,chicken american (traditional) sauce restauran...
47 Scott,american (traditional) mealrestaurants gastrop...
5 Points Market & Restaurant,mealrestaurants time back place sauce great lo...
99 Cents Only Stores,event planning & services grocery discount sto...


**Count Vectorizer**

In [58]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(az_final['bag_of_words'])

# creating a Series for the restaurant names so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(az_final.index)
indices[:5]

0                            1702
1                       4 Seasons
2                        47 Scott
3    5 Points Market & Restaurant
4            99 Cents Only Stores
Name: name, dtype: object

In [59]:
count_matrix.shape

(393, 427)

In [60]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.07784989, 0.46188022, ..., 0.36514837, 0.21773242,
        0.6846532 ],
       [0.07784989, 1.        , 0.67419986, ..., 0.51168172, 0.38138504,
        0.2665009 ],
       [0.46188022, 0.67419986, 1.        , ..., 0.63245553, 0.42426407,
        0.671984  ],
       ...,
       [0.36514837, 0.51168172, 0.63245553, ..., 1.        , 0.38758512,
        0.5       ],
       [0.21773242, 0.38138504, 0.42426407, ..., 0.38758512, 1.        ,
        0.2236068 ],
       [0.6846532 , 0.2665009 , 0.671984  , ..., 0.5       , 0.2236068 ,
        1.        ]])

In [66]:
# function that takes in movie title as input and returns the top 10 recommended movies
def content_based_recommendations(name, cosine_sim = cosine_sim):

    recommended_restaurants = []

    # gettin the index of the movie that matches the title
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)

    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_restaurants.append(list(az_final.index)[i])

    return recommended_restaurants

**Examples**

In [70]:
content_based_recommendations('eHotPot')

['China Szechwan',
 'Taqueria Pico De Gallo',
 'Tamarind',
 'El Torero',
 "Taqueria Juanito's",
 "Sullivan's Steakhouse",
 'Wok & Roll',
 'Chef Wang',
 'Sushi Cortaro',
 'Com Tam Thuan Kieu']

In [71]:
restaurant = az_data[az_data['name'] == 'eHotPot'][['name','categories','super_score', 'All_Keywords']]
restaurant.head(1)

,name,categories,super_score,All_Keywords
140,eHotPot,"Chinese, Taiwanese, Hot Pot, Restaurants",5,"chinese, taiwanese, hot pot, restaurants, orde..."


In [72]:
restaurant1 = az_data[az_data['name'] == 'China Szechwan'][['name','categories','super_score', 'All_Keywords']]
restaurant1.head(1)

,name,categories,super_score,All_Keywords
132,China Szechwan,"Restaurants, Chinese, Food, Szechuan",4,"restaurants, chinese, food, szechuan, order, f..."


In [75]:
content_based_recommendations('5 Points Market & Restaurant')

["Sullivan's Steakhouse",
 "Inca's Peruvian Cuisine",
 'Com Tam Thuan Kieu',
 'China Szechwan',
 'GRINGO grill + cantina',
 'Bottega Michelangelo',
 '47 Scott',
 'Bisbee Breakfast Club',
 'R&R Bar And Grill',
 'Barista Del Barrio']

**Based on these case studies, we observe that our content-based recommendation system is more able to recommend similar restaurants based on categories and type of food served, than collaborative filtering based recommendation system.**